In [ ]:
from fastai.vision.all import *

path = untar_data(URLs.IMAGENETTE)

In [ ]:
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=aug_transforms(size=224, min_scale=0.75)
                   )

In [ ]:
dls = dblock.dataloaders(path, bs=64)

In [ ]:
model = xresnet50()

In [ ]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 

In [ ]:
### as its not transfer learning we use some techniques
#### normalization

x, y = dls.one_batch()
x.mean(dim=[0,2,3]), x.std(dim=[0, 2, 3])

In [ ]:
## bs = batch size and size=image size

def get_dls(bs, size):
 dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
 get_items=get_image_files,
 get_y=parent_label,
 item_tfms=Resize(460),
 batch_tfms=[*aug_transforms(size=size, min_scale=0.75),
 Normalize.from_stats(*imagenet_stats)])
 return dblock.dataloaders(path, bs=bs)

dls = get_dls(64, 224)

In [ ]:
x, y = dls.one_batch()
x.mean(dim=[0,2,3]), x.std(dim=[0, 2, 3])

In [ ]:
### check the effect of the normalization

model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

In [ ]:
dls = get_dls(128, 128)
learn = Learner(dls, xresnet50(), loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(4, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.854688,3.133317,0.284167,1:01:17


In [ ]:
learn.dls = get_dls(64, 224)
learn.fine_tune(5, 1e-3)

In [ ]:
preds, targs = learn.tta()
accuracy(preds, targs).item()

In [ ]:
model = xresnet50()
learn = Learner(dls, model, loss_func=LabelSmothingCrossEntropy(), metrics=accuracy)
learn.freeze()